In [1]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()

  Activating project at `c:\Users\Reintjes\Documents\aD\UoC\Semester1\AMO\Projekt\AMOProject`


In [2]:
import Pkg; Pkg.add("JuMP")
Pkg.add("Distributions")
Pkg.add("HiGHS")
Pkg.add("Clustering")
Pkg.add("Distances")
Pkg.add("Plots")
Pkg.add("Statistics")
Pkg.add("StatsBase")
Pkg.add("Ipopt")

using JuMP, Distributions, HiGHS, Clustering, Distances, Plots, Statistics, StatsBase, Ipopt
gr()

   Resolving package versions...
  No Changes to `C:\Users\Reintjes\Documents\aD\UoC\Semester1\AMO\Projekt\AMOProject\Project.toml`
  No Changes to `C:\Users\Reintjes\Documents\aD\UoC\Semester1\AMO\Projekt\AMOProject\Manifest.toml`
   Resolving package versions...
   Installed PDMats ────────────────── v0.11.32
   Installed PtrArrays ─────────────── v1.3.0
   Installed QuadGK ────────────────── v2.11.2
   Installed HypergeometricFunctions ─ v0.3.27
   Installed Distributions ─────────── v0.25.117
    Updating `C:\Users\Reintjes\Documents\aD\UoC\Semester1\AMO\Projekt\AMOProject\Project.toml`
  [31c24e10] + Distributions v0.25.117
    Updating `C:\Users\Reintjes\Documents\aD\UoC\Semester1\AMO\Projekt\AMOProject\Manifest.toml`
  [66dad0bd] + AliasTables v1.1.3
  [9a962f9c] + DataAPI v1.16.0
  [31c24e10] + Distributions v0.25.117
  [1a297f60] + FillArrays v1.13.0
  [34004b35] + HypergeometricFunctions v0.3.27
  [e1d29d7a] + Missings v1.2.0
  [90014a1f] + PDMats v0.11.32
  [43287f4e] + PtrA

Plots.GRBackend()

In [3]:
Pkg.add("BilevelJuMP")

using BilevelJuMP

   Resolving package versions...
   Installed Dualization ─ v0.5.8
   Installed BilevelJuMP ─ v0.6.2
    Updating `C:\Users\Reintjes\Documents\aD\UoC\Semester1\AMO\Projekt\AMOProject\Project.toml`
  [485130c0] + BilevelJuMP v0.6.2
    Updating `C:\Users\Reintjes\Documents\aD\UoC\Semester1\AMO\Projekt\AMOProject\Manifest.toml`
  [485130c0] + BilevelJuMP v0.6.2
  [191a621a] + Dualization v0.5.8
Precompiling project...
   5599.3 ms  ✓ Dualization
   5653.3 ms  ✓ BilevelJuMP
  2 dependencies successfully precompiled in 17 seconds. 233 already precompiled.


In [4]:
# Generate 288 distribution vectors
function generate_distributions(si::Float64, consumption_mean::Float64, temp_mean::Float64, hour::Int)
    if hour < 6 || hour > 22
        solar = Beta(1, 1000)  # Close to zero but keeps Beta distribution structure
        temperature = Normal(temp_mean-4, 9)
        consumption = Normal(consumption_mean/(consumption_mean-1), consumption_mean / 11)    
    else
        β = ((si * (1 + si) / (si / 10)) - 1) * (1 - si)
        α = si * β / (1 - si)
        solar = Beta(α, β)
        temperature = Normal(temp_mean, 9)
        consumption = Normal(consumption_mean, consumption_mean / 11)    

    end
    return [solar, consumption, temperature]
end

dist_vec = Vector{Vector{Distribution}}(undef, 12 * 24)

for month in 1:12
    for hour in 1:24
        # Assume some seasonal variations
        si = 0.25 + 0.1 * sin(2π * month / 12)  # Varying solar irradiance
        si = hour < 6 || hour > 22 ? 0.0 : si   # Zero irradiance at night
        consumption_mean = 90.0 + 10 * sin(2π * hour / 24)  # Daily load cycle
        temp_mean = 13.0 + 5 * cos(2π * month / 12)  # Seasonal temperature
        
        # Store the distribution vector
        dist_vec[(month - 1) * 24 + hour] = generate_distributions(si, consumption_mean, temp_mean, hour)
    end
end

# Check an example
dist_vec[1]  # First hour of January

3-element Vector{Distribution}:
 Beta{Float64}(α=1.0, β=1000.0)
 Normal{Float64}(μ=1.0109184382295962, σ=8.417108222820474)
 Normal{Float64}(μ=13.330127018922195, σ=9.0)

In [5]:
# Function to sample, cluster, and compute probabilities
function cluster_scenarios_with_probabilities(dist_vec, num_samples=5000, num_clusters=5)
    scenarios = []
    probabilities = []
    
    for (i, dist_set) in enumerate(dist_vec)
        # Generate 5000 samples for each variable
        samples = hcat(rand(dist_set[1], num_samples),  # Solar Irradiance
                       rand(dist_set[2], num_samples),  # Energy Consumption
                       rand(dist_set[3], num_samples)) # Temperature
        
        # Ensure non-negativity
        samples[:, 1] .= max.(samples[:, 1], 0)  # Solar Irradiance ≥ 0
        samples[:, 2] .= max.(samples[:, 2], 0)  # Consumption ≥ 0

        # Apply k-means clustering
        result = kmeans(samples', num_clusters)  # Clustering (transpose for correct shape)
        centroids = result.centers'  # Get cluster centers (transpose back)
        
        # Compute probabilities (relative frequency of points in each cluster)
        cluster_counts = countmap(result.assignments)  # Count points per cluster
        probs = [cluster_counts[j] / num_samples for j in 1:num_clusters]  # Normalize

        # Store results
        push!(scenarios, centroids)
        push!(probabilities, probs)
    end
    
    return scenarios, probabilities
end

# Generate 5 scenarios and their probabilities for all 288 time periods
scenarios_5, scenario_probs = cluster_scenarios_with_probabilities(dist_vec)

# Check an example
println("Scenarios for January, 1st hour: ", scenarios_5[1]) 
println("Probabilities for January, 1st hour: ", scenario_probs[1], " sum ", sum(scenario_probs[1]))  

Scenarios for January, 1st hour: [0.0010181883321161937 1.250611057864984 13.130000188223073; 0.0010040494021688203 1.3805541273468471 24.24339678535391; 0.0010114403319835404 12.118820349492545 8.243961186464318; 0.000991840336059702 1.7986412055442285 1.5215513015201922; 0.0008836223594330942 11.765655982026674 20.8655477381091]
Probabilities for January, 1st hour: [0.3578, 0.2104, 0.1216, 0.2096, 0.1006] sum 1.0


## Optimization Model

### Scalars

In [63]:
T = 288  # 12 months * 24 hours
num_scenarios = 5  # 5 clusters per time step

SCC = 0  # Social cost of carbon ($/ton CO2)
# E_cap = 1000 Emission cap
C_inv = 10 * 1  # Investment cost (per day for 12 days per PV module)
C_op = 0,1 * 12 # operation cost per day for 12 days

c_grid_buy = 0.05  # Cost of buying from grid ($/kWh)
c_grid_feed = 0.10  # Revenue from feeding into the grid ($/kWh)
e_grid = 0.5  # Emission intensity of grid (tCO2/kWh)

# PV Parameters
η_0 = 0.18  # Nominal PV efficiency at 25°C
β = 0.004  # Temperature coefficient (per °C)
T_ref = 25  # Reference temperature (°C)
A = 1  # Area of PV system (arbitrary unit, related to y_PV)

function efficiency(T_t)
    return η_0 * (1 - β * (T_t - T_ref))  # Efficiency function
end


# Solar irradiance: column 1
S_t_omega = [[scenarios_5[t][ω, 1] for ω in 1:num_scenarios] for t in 1:T]

# Consumption: column 2
L_t_omega = [[scenarios_5[t][ω, 2] for ω in 1:num_scenarios] for t in 1:T]

# Temperature: column 3
T_t_omega = [[scenarios_5[t][ω, 3] for ω in 1:num_scenarios] for t in 1:T]

# Scenario probabilities
Φ_t_omega = [[scenario_probs[t][ω] for ω in 1:num_scenarios] for t in 1:T] 

288-element Vector{Vector{Float64}}:
 [0.3578, 0.2104, 0.1216, 0.2096, 0.1006]
 [0.2038, 0.2042, 0.1156, 0.38, 0.0964]
 [0.355, 0.2134, 0.1176, 0.2104, 0.1036]
 [0.2054, 0.3578, 0.2102, 0.108, 0.1186]
 [0.2102, 0.1222, 0.113, 0.357, 0.1976]
 [0.1982, 0.1762, 0.1714, 0.2584, 0.1958]
 [0.1538, 0.216, 0.1298, 0.278, 0.2224]
 [0.2552, 0.178, 0.1978, 0.1948, 0.1742]
 [0.1824, 0.1826, 0.2642, 0.228, 0.1428]
 [0.2436, 0.1948, 0.1644, 0.1412, 0.256]
 ⋮
 [0.214, 0.148, 0.2582, 0.2162, 0.1636]
 [0.22, 0.1672, 0.2648, 0.2086, 0.1394]
 [0.1576, 0.1544, 0.2218, 0.1986, 0.2676]
 [0.1538, 0.2832, 0.2088, 0.2012, 0.153]
 [0.1358, 0.1526, 0.2792, 0.2124, 0.22]
 [0.1416, 0.1638, 0.2394, 0.256, 0.1992]
 [0.2756, 0.204, 0.157, 0.1462, 0.2172]
 [0.1124, 0.2064, 0.1958, 0.3556, 0.1298]
 [0.1612, 0.2696, 0.1372, 0.2808, 0.1512]

### Model

In [70]:
model = Model(Ipopt.Optimizer)

# Upper Level variables
@variable(model, 0 <= y_PV <= 30)  # PV Capacity and upper bound

# Lower Level Variables (Operational Dispatch)
@variable(model, G_in[t=1:T, ω=1:num_scenarios] >= 0)  # Grid import
@variable(model, G_out[t=1:T, ω=1:num_scenarios] >= 0)  # Grid feed-in
@variable(model, P_PV[t=1:T, ω=1:num_scenarios] >= 0)  # PV production

# Meeting the demand
@constraint(model, [t=1:T, ω=1:num_scenarios],
    L_t_omega[t][ω] == y_PV * P_PV[t, ω] + G_in[t, ω] - G_out[t, ω]
)
# Lower Level Objective: Minimize Operating Cost
@expression(model, C_op,
    sum(Φ_t_omega[t][ω] * (c_grid_buy * G_in[t, ω] - c_grid_feed * G_out[t, ω]) 
        for t in 1:T, ω in 1:num_scenarios)
)

# PV production is calculated based on efficiency function
@constraint(model, [t=1:T, ω=1:num_scenarios],
    P_PV[t, ω] == efficiency(T_t_omega[t][ω]) * A * S_t_omega[t][ω]
)

# Emissions Calculation
@expression(model, E_op,
    sum(Φ_t_omega[t][ω] * e_grid * G_in[t, ω] for t in 1:T, ω in 1:num_scenarios)
)

# Upper Level Objective: Minimize Total Cost
@objective(model, Min,
    C_inv * y_PV # Investment Cost + Operating Cost
    + C_op  # Expected Operating Cost
    + SCC * E_op  # Expected Emissions Cost
)

10 y_PV + 0.01789 G_in[1,1] - 0.03578 G_out[1,1] + 0.010520000000000002 G_in[1,2] - 0.021040000000000003 G_out[1,2] + 0.00608 G_in[1,3] - 0.01216 G_out[1,3] + 0.010480000000000001 G_in[1,4] - 0.020960000000000003 G_out[1,4] + 0.00503 G_in[1,5] - 0.01006 G_out[1,5] + 0.010190000000000001 G_in[2,1] - 0.020380000000000002 G_out[2,1] + 0.01021 G_in[2,2] - 0.02042 G_out[2,2] + 0.00578 G_in[2,3] - 0.01156 G_out[2,3] + 0.019000000000000003 G_in[2,4] - 0.038000000000000006 G_out[2,4] + 0.0048200000000000005 G_in[2,5] - 0.009640000000000001 G_out[2,5] + 0.01775 G_in[3,1] - 0.0355 G_out[3,1] + 0.01067 G_in[3,2] - 0.02134 G_out[3,2] + 0.00588 G_in[3,3] - 0.01176 G_out[3,3] + 0.010520000000000002 G_in[3,4] - 0.021040000000000003 G_out[3,4] + 0.005180000000000001 G_in[3,5] - [[...2821 terms omitted...]] + 0.01378 G_in[286,1] - 0.02756 G_out[286,1] + 0.0102 G_in[286,2] - 0.0204 G_out[286,2] + 0.007850000000000001 G_in[286,3] - 0.015700000000000002 G_out[286,3] + 0.0073100000000000005 G_in[286,4] - 0

### Solving

In [71]:
optimize!(model)


This is Ipopt version 3.14.17, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:     7200
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:     1440

Total number of variables............................:     4321
                     variables with only lower bounds:     4320
                variables with lower and upper bounds:        1
                     variables with only upper bounds:        0
Total number of equality constraints.................:     2880
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 -4.3999956e-02 1.13e+02 9.13e+00  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

In [72]:
y_PV_value = value(y_PV)
println("Optimal PV Capacity (y_PV): $y_PV_value")

Optimal PV Capacity (y_PV): 0.01054949736395016


## Bi Level Opt

In [74]:
using BilevelJuMP
model = BilevelModel()


An Abstract JuMP Model
Feasibility problem with:
Variables: 0
Upper Constraints: 0
Lower Constraints: 0
Bilevel Model
Solution method: BilevelJuMP.NoMode{Float64}
No solver attached

In [75]:
#--------------------------------------------------------------------
# UPPER
#--------------------------------------------------------------------
@variable(Upper(model), 0 <= y_PV <= 30)
@variable(Lower(model), y_PV_lower >= 0)
@constraint(Upper(model), link_yPV, y_PV_lower == y_PV)

# Summenvariablen in der Upper-Ebene (Kopie der Lower-Ergebnisse)
@variable(Upper(model), cost_op_upper >= 0)  # operative Kosten
@variable(Upper(model), E_op_upper   >= 0)  # Emissionen

# Koppel-Constraints für die Summenvariablen, 
# die wir gleich in der Lower-Ebene definieren:
@variable(Lower(model), cost_op_lower >= 0)
@variable(Lower(model), E_op_lower    >= 0)

@constraint(Upper(model), link_cost, cost_op_upper == cost_op_lower)
@constraint(Upper(model), link_emis, E_op_upper   == E_op_lower)

# Oberes Objective: Investitionskosten + cost_op_upper + SCC * E_op_upper
@objective(Upper(model), Min,
    C_inv*y_PV
    + cost_op_upper
    + SCC*E_op_upper
)


#--------------------------------------------------------------------
# LOWER
#--------------------------------------------------------------------
@variable(Lower(model), G_in[t=1:T, ω=1:num_scenarios] >= 0)
@variable(Lower(model), G_out[t=1:T, ω=1:num_scenarios] >= 0)

# Hilfs-Ausdruck: Einstrahlfaktor (keine Variable!)
@expression(Lower(model),
    alpha[t in 1:T, ω in 1:num_scenarios],
    efficiency(T_t_omega[t][ω]) * A * S_t_omega[t][ω]
)

# BilanzConstraint -> y_PV_lower statt y_PV!
@constraint(Lower(model), [t=1:T, ω in 1:num_scenarios],
    L_t_omega[t][ω] == (y_PV_lower * alpha[t,ω]) + G_in[t,ω] - G_out[t,ω]
)

# Lower-Objective: Minimierung der operativen Kosten
@objective(Lower(model), Min,
    sum(
        Φ_t_omega[t][ω]*(
            c_grid_buy*G_in[t,ω] - c_grid_feed*G_out[t,ω]
        ) for t in 1:T, ω in 1:num_scenarios
    )
)

# ABER wir wollen, dass "cost_op_lower" diese Summe auch repräsentiert:
@constraint(Lower(model),
    cost_op_link,
    cost_op_lower 
     == 
    sum(Φ_t_omega[t][ω]*(
            c_grid_buy*G_in[t,ω] - c_grid_feed*G_out[t,ω]
        ) for t in 1:T, ω in 1:num_scenarios)
)

# Genauso für die Emissionen:
@constraint(Lower(model),
    emis_link,
    E_op_lower
     ==
    sum(Φ_t_omega[t][ω]*e_grid*G_in[t,ω]
        for t in 1:T, ω in 1:num_scenarios)
)


emis_link : E_op_lower - 0.1789 G_in[1,1] - 0.1019 G_in[2,1] - 0.1775 G_in[3,1] - 0.1027 G_in[4,1] - 0.1051 G_in[5,1] - 0.0991 G_in[6,1] - 0.0769 G_in[7,1] - 0.1276 G_in[8,1] - 0.0912 G_in[9,1] - 0.1218 G_in[10,1] - 0.0947 G_in[11,1] - 0.07 G_in[12,1] - 0.0794 G_in[13,1] - 0.1382 G_in[14,1] - 0.1021 G_in[15,1] - 0.0997 G_in[16,1] - 0.0979 G_in[17,1] - 0.1066 G_in[18,1] - 0.1238 G_in[19,1] - 0.097 G_in[20,1] - 0.1049 G_in[21,1] - 0.1372 G_in[22,1] - 0.0692 G_in[23,1] - 0.1017 G_in[24,1] - 0.1426 G_in[25,1] - 0.1364 G_in[26,1] - 0.1035 G_in[27,1] - 0.0945 G_in[28,1] - 0.0891 G_in[29,1] - [[...1381 terms omitted...]] - 0.115 G_in[259,5] - 0.0872 G_in[260,5] - 0.1308 G_in[261,5] - 0.1043 G_in[262,5] - 0.0766 G_in[263,5] - 0.0756 G_in[264,5] - 0.0718 G_in[265,5] - 0.0559 G_in[266,5] - 0.1804 G_in[267,5] - 0.06 G_in[268,5] - 0.0579 G_in[269,5] - 0.0768 G_in[270,5] - 0.1273 G_in[271,5] - 0.1031 G_in[272,5] - 0.1032 G_in[273,5] - 0.1118 G_in[274,5] - 0.1232 G_in[275,5] - 0.0965 G_in[276,5] - 0

In [76]:
set_optimizer(model, Ipopt.Optimizer)

An Abstract JuMP Model
Minimization problem with:
Variables: 2886
Upper objective function type: 
GenericAffExpr{Float64, BilevelVariableRef}
Lower objective function type: 
GenericAffExpr{Float64, BilevelVariableRef}
Upper Constraints: 3
Lower Constraints: 1442
Bilevel Model
Solution method: BilevelJuMP.NoMode{Float64}
Solver name: Ipopt
Names registered in the model: E_op_lower, E_op_upper, G_in, G_out, alpha, cost_op_link, cost_op_lower, cost_op_upper, emis_link, link_cost, link_emis, link_yPV, y_PV, y_PV_lower

In [77]:
BilevelJuMP.set_mode(model,
BilevelJuMP.ProductMode())

An Abstract JuMP Model
Minimization problem with:
Variables: 2886
Upper objective function type: 
GenericAffExpr{Float64, BilevelVariableRef}
Lower objective function type: 
GenericAffExpr{Float64, BilevelVariableRef}
Upper Constraints: 3
Lower Constraints: 1442
Bilevel Model
Solution method: BilevelJuMP.ProductMode{Float64}(0.0, false, 0, nothing)
Solver name: Ipopt
Names registered in the model: E_op_lower, E_op_upper, G_in, G_out, alpha, cost_op_link, cost_op_lower, cost_op_upper, emis_link, link_cost, link_emis, link_yPV, y_PV, y_PV_lower

In [78]:
optimize!(model)


This is Ipopt version 3.14.17, running with linear solver MUMPS 5.7.3.

Number of nonzeros in equality constraint Jacobian...:    20173
Number of nonzeros in inequality constraint Jacobian.:     5766
Number of nonzeros in Lagrangian Hessian.............:     2883

Total number of variables............................:     7211
                     variables with only lower bounds:     5768
                variables with lower and upper bounds:        1
                     variables with only upper bounds:        0
Total number of equality constraints.................:     4328
Total number of inequality constraints...............:     2883
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:     2883

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  1.0999989e-01 1.13e+02 1.05e+01  -1.0 0.00e+00    -  0.00e+00 0.00e+00 

In [69]:
y_PV_value = value(y_PV)
println("Optimal PV Capacity (y_PV): $y_PV_value")

Optimal PV Capacity (y_PV): 17.72429451422677


In [81]:
println("Upper: y_PV =", value(y_PV))
println("Upper: cost_op_upper =", value(cost_op_upper))
println("Upper: E_op_upper   =", value(E_op_upper))
println("Obj =", objective_value(model))

println("Lower: y_PV =", value(y_PV_lower))
println("Upper: cost_op_upper =", value(cost_op_lower))
println("Upper: E_op_upper   =", value(E_op_lower))

Upper: y_PV =17.72429451422677
Upper: cost_op_upper =901.1337498537979
Upper: E_op_upper   =9143.80712595572
Obj =1078.3766949960657
Lower: y_PV =17.72429451422677
Upper: cost_op_upper =901.1337498537979
Upper: E_op_upper   =9143.80712595572
